# Implementation of GoogLeNet using Google resources

We will introduce GoogLeNet, presenting a slightly simplified version of the original model: we omit a few ad-hoc features that were added to stabilize training but are unnecessary now with better training algorithms available.

In [1]:
#Necessary to run in notebook
!pip install d2l==0.17.6

     |████████████████████████████████| 112 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 16.9 MB 7.2 MB/s eta 0:00:01     |█████▏                          | 2.7 MB 10.8 MB/s eta 0:00:02
     |████████████████████████████████| 7.3 MB 87 kB/s eta 0:00:0101
     |████████████████████████████████| 61 kB 7.1 MB/s eta 0:00:011
     |████████████████████████████████| 10.5 MB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 5.9 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.2
    Uninstalling matplotlib-3.3.2:
      Successfully uninstalled matplotlib-3.3.2
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
  Attempting uninstall: pandas
    Foun

## Define Inception Blocks

In [6]:
import tensorflow as tf
#from d2l import tensorflow as d2l

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf
from d2l import tensorflow as d2l


class Inception(tf.keras.Model):
    # `c1`--`c4` are the number of output channels for each path
    def __init__(self, c1, c2, c3, c4):
        super().__init__()
        # Path 1 is a single 1 x 1 convolutional layer
        self.p1_1 = tf.keras.layers.Conv2D(c1, 1, activation='relu')
        # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3
        # convolutional layer
        self.p2_1 = tf.keras.layers.Conv2D(c2[0], 1, activation='relu')
        self.p2_2 = tf.keras.layers.Conv2D(c2[1], 3, padding='same',
                                           activation='relu')
        # Path 3 is a 1 x 1 convolutional layer followed by a 5 x 5
        # convolutional layer
        self.p3_1 = tf.keras.layers.Conv2D(c3[0], 1, activation='relu')
        self.p3_2 = tf.keras.layers.Conv2D(c3[1], 5, padding='same',
                                           activation='relu')
        # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1
        # convolutional layer
        self.p4_1 = tf.keras.layers.MaxPool2D(3, 1, padding='same')
        self.p4_2 = tf.keras.layers.Conv2D(c4, 1, activation='relu')


    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        # Concatenate the outputs on the channel dimension
        return tf.keras.layers.Concatenate()([p1, p2, p3, p4])

## Implement GoogLeNet module by module

### Block 1

In [ ]:
# The first module uses a 64-channel  7×7  convolutional layer.
def b1():
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, 7, strides=2, padding='same',
                               activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

### Block 2:

In [ ]:
# The second module uses two convolutional layers: first, a 64-channel  1×1  convolutional layer, then a  3×3  
# convolutional layer that triples the number of channels. This corresponds to the second path in the Inception block.
def b2():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, 1, activation='relu'),
        tf.keras.layers.Conv2D(192, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

### Block 3:

In [ ]:
# The third module connects two complete Inception blocks in series.
# Maximum pooling between inception blocks reduces the dimensionality.
def b3():
    return tf.keras.models.Sequential([
        Inception(64, (96, 128), (16, 32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

### Block 4:

In [ ]:
# The fourth module is more complicated. It connects five Inception blocks in series
# Maximum pooling between inception blocks reduces the dimensionality.
def b4():
    return tf.keras.Sequential([
        Inception(192, (96, 208), (16, 48), 64),
        Inception(160, (112, 224), (24, 64), 64),
        Inception(128, (128, 256), (24, 64), 64),
        Inception(112, (144, 288), (32, 64), 64),
        Inception(256, (160, 320), (32, 128), 128),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

### Block 5:

In [ ]:
# The fifth module has two Inception blocks and is followed by the output layer.
# This block uses the global average pooling layer to change the height and width of each channel to 1
# turn the output into a two-dimensional array followed by a fully-connected layer whose number of outputs is the number of label classes.
def b5():
    return tf.keras.Sequential([
        Inception(256, (160, 320), (32, 128), 128),
        Inception(384, (192, 384), (48, 128), 128),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Flatten()
    ])
# Recall that this has to be a function that will be passed to
# `d2l.train_ch6()` so that model building/compiling need to be within
# `strategy.scope()` in order to utilize the CPU/GPU devices that we have

# Define final model:

In [ ]:
# Change Outout layer to number of label classes!
def googlenet():
    return tf.keras.Sequential([b1(), b2(), b3(), b4(), b5(),
                                tf.keras.layers.Dense(10)])